# Merge vs Join in Pandas

## data

In [1]:
import pandas as pd
import io

csv = '''
salary,employee_id
2000,2
2500,3
2200,4
'''

csv2 = '''
employee_id,seniority
1,0ans
2,2ans
3,4ans
'''


salaries = pd.read_csv(io.StringIO(csv))
seniorities = pd.read_csv(io.StringIO(csv2))

In [2]:
salaries

,salary,employee_id
0,2000,2
1,2500,3
2,2200,4


In [3]:
seniorities

,employee_id,seniority
0,1,0ans
1,2,2ans
2,3,4ans


## test join:

In [4]:
salaries.join(seniorities,
              on="employee_id",
              how="inner",
              lsuffix="_")  # sans lsuffix, la méthode va fail

,salary,employee_id_,employee_id,seniority
0,2000,2,3,4ans


Mais pourquoi on obtient un résultat aberrant comme celui-ci ?

On lui demande d'utiliser la colonne employee_id. Donc pandas va chercher dans la table seniorities
des valeurs pour 2-3-4:

In [5]:
salaries

,salary,employee_id
0,2000,2
1,2500,3
2,2200,4


Mais il ne va pas les chercher dans la colonne employee_id de seniorities, il va les chercher dans l'index !

Or, l'index (les chiffres sur la gauche de la dataframe) de seniorities s'arrête à 2!

In [6]:
new_row = pd.DataFrame(
    {"employee_id": "who cares ?",
     "seniority": "doesn't matter"},
    index=[4]  # on rajoute une ligne qui contient un index 4 pour matcher avec l'employee_id=4 de la table salaries
)
seniorities_four_lines = pd.concat([seniorities, new_row])
seniorities_four_lines

,employee_id,seniority
0,1,0ans
1,2,2ans
2,3,4ans
4,who cares ?,doesn't matter


In [7]:
salaries.join(
    seniorities_four_lines,
    on="employee_id",
    how="inner",
    lsuffix="_",
)  

,salary,employee_id_,employee_id,seniority
0,2000,2,3,4ans
2,2200,4,who cares ?,doesn't matter


<br />

<br />
<br />

Conclusion: la méthode .join() utilise les index pour faire les jointures.


## Joindre sur une colonne en utilisant .join()



Si on veut joindre sur une colonne, il faut mettre celle-ci en index:

In [8]:
seniorities_idx = seniorities.set_index("employee_id")
seniorities_idx

,seniority
employee_id,
1,0ans
2,2ans
3,4ans


In [9]:
salaries_idx = salaries.set_index("employee_id")
salaries_idx

,salary
employee_id,
2,2000
3,2500
4,2200


In [10]:
seniorities_idx.join(salaries_idx, how="inner")

,seniority,salary
employee_id,,
2,2ans,2000
3,4ans,2500


## Joindre sur une colonne en utilisant merge

<img src="images/merge_chandler.gif" />

Documentation pandas pour .merge(): 

<blockquote>Merge DataFrame or named Series objects with a <i>database-style <b><u>join</u></b></i>. </blockquote>

In [11]:
salaries.merge(seniorities, on="employee_id", how="inner")

,salary,employee_id,seniority
0,2000,2,2ans
1,2500,3,4ans


Pourquoi ils ont pas décidé de mettre le "database-style join" dans .join() plutôt que dans .merge() ? 

Honnêtement aucune idée; Personnellement je trouve que c'est un défaut d'API qui fait buguer les débutants. Mais bon, c'est comme ça donc il faut le savoir ;)

Conclusion: merge permet de joindre en utilisant une colonne, join demande à ce que la "colonne" utilisée pour joindre soit l'index. L'immense majorité du temps, j'utilise merge parce que c'est plus intuitif pour moi.